In [6]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib
import gc
gc.enable() 



def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='taiwanInfoIta'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

mailTo(title="taiwanInfoItabegin",mailAdds=["andy.yuan@wowprime.com"],message="分析開始")
try:
    ######################################################第一次跑用stoneTwo來篩選沒超商的點
    b=time.time()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collectionTaiwan = client.rawData.taiwanInfoMaBoss
#     collectionTaiwan = client.rawData.taiwanInfoIta
    TWLocs=list(collectionTaiwan.find({}, {'_id': False}))
    
    ######################################################IPEEN
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find({}, {'_id': False}))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0]

    ######################################################IPEEN
    ######################################################conStore0320
    collection = client.rawData.infoClinic
    clinicData = list(collection.find({}, {'_id': False}))
    ######################################################conStore0320
    ######################################################Watsons
    collection = client.rawData.dahu
    dahuData = list(collection.find({}, {'_id': False}))
    ######################################################Watsons
    ######################################################canSuMe
    collection = client.rawData.primarySchool
    pschData = list(collection.find({}, {'_id': False}))
    ######################################################canSuMe
    ##################################################第一次跑先去除latlng以外的資訊
    TWLocs = [{"lat":i["lat"],"lng":i["lng"]} for i in TWLocs if ('NconStore_Analyze' in i and
              i['NconStore_Analyze'] >0)]
#     TWLocs = [i for i in TWLocs if ('NconStore_Analyze' in i and
#               i['NconStore_Analyze'] >0)]
    ##################################################第一次跑先去除latlng以外的資訊
    allN=len(TWLocs)
    print("有{}筆資料要分析".format(allN))
    radius=500
    n = 0

    for TWLoc in TWLocs:
        score=0
        n += 1
        if n % 50 == 0:
            e=time.time()
            print("已完成{}%分析-{}項,花費{}秒".format(round(n/allN,4)*100,n,round(e-b)))
        if n % 2000 == 0:
            gc.collect()
            client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfoIta
            
            ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            datas=[i for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,datas)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()

        TWLoc["areaRadius_Analyze"] = radius
        #############################################################0523Score
        #+3
        TWLoc["Nclinic_Analyze"] = len([dien['name'] for dien in clinicData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius])
            
        #+3
        TWLoc["Ndahu_Analyze"] = len([dien for dien in dahuData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        #+1
        TWLoc["Npsch_Analyze"] = len([dien for dien in pschData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= 200])
        

        rangeCost = 1.4
        
        #+2
        simDien = [dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['averagecost'] < ((340) * rangeCost)
                   and dien['averagecost'] > ((340) / rangeCost)]
        
        TWLoc["NgoodDien"] = len(simDien)
        
        #-1
        TWLoc["Nstar_Analyze"] = len([dien['name'] for dien in ipeendata 
                                if '星巴' in dien['name'] 
                                and haversine(lng1=dien["lng"],
                                              lat1=dien["lat"],
                                              lng2=TWLoc["lng"],
                                              lat2=TWLoc["lat"]) <= radius])
        

        #############################################################0523Score
    for i in TWLocs:
        score=0

        if i['Nclinic_Analyze']>=21:
            score+=3
        elif i['Nclinic_Analyze']>=14:
            score+=2
        elif i['Nclinic_Analyze']>=9:
            score+=1

        if i['Ndahu_Analyze']>=1:
            score+=1

        if i['NgoodDien']>=23:
            score+=3
        elif i['NgoodDien']>=17:
            score+=2
        elif i['NgoodDien']>=10:
            score+=1


        if i['Npsch_Analyze']>=1:
            score-=1

        if i['Nstar_Analyze']>=3:
            score+=3
        elif i['Nstar_Analyze']>=2:
            score+=2
        elif i['Nstar_Analyze']>=1:
            score+=1

        i['score']=score
        
    
    e = time.time()


#     client.close()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collection = client.rawData.taiwanInfoIta
    ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs]
    operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
    try:
        collection.bulk_write(operations ,ordered=False)
    except Exception as er:
        print(er)
    client.close()
    
    e = time.time()
    print(e-b)
    
    
    mailTo(title="taiwanInfoItaSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b))
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="taiwanInfoItaFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg)

有50671筆資料要分析
已完成0.1%分析-50項,花費33秒
已完成0.2%分析-100項,花費60秒
已完成0.3%分析-150項,花費86秒
已完成0.38999999999999996%分析-200項,花費112秒
已完成0.49%分析-250項,花費138秒
已完成0.59%分析-300項,花費164秒
已完成0.69%分析-350項,花費191秒
已完成0.79%分析-400項,花費220秒
已完成0.89%分析-450項,花費249秒
已完成0.9900000000000001%分析-500項,花費278秒
已完成1.09%分析-550項,花費306秒
已完成1.18%分析-600項,花費332秒
已完成1.28%分析-650項,花費358秒
已完成1.38%分析-700項,花費385秒
已完成1.48%分析-750項,花費412秒
已完成1.58%分析-800項,花費440秒
已完成1.68%分析-850項,花費470秒
已完成1.78%分析-900項,花費501秒
已完成1.87%分析-950項,花費530秒
已完成1.97%分析-1000項,花費561秒
已完成2.07%分析-1050項,花費590秒
已完成2.17%分析-1100項,花費619秒
已完成2.27%分析-1150項,花費652秒
已完成2.37%分析-1200項,花費685秒
已完成2.4699999999999998%分析-1250項,花費716秒
已完成2.5700000000000003%分析-1300項,花費747秒
已完成2.6599999999999997%分析-1350項,花費777秒
已完成2.76%分析-1400項,花費808秒
已完成2.86%分析-1450項,花費838秒
已完成2.96%分析-1500項,花費868秒
已完成3.06%分析-1550項,花費899秒
已完成3.16%分析-1600項,花費931秒
已完成3.26%分析-1650項,花費967秒
已完成3.35%分析-1700項,花費1002秒
已完成3.45%分析-1750項,花費1033秒
已完成3.55%分析-1800項,花費1067秒
已完成3.65%分析-1850項,花費1100秒
已完成3.75%分析-1900項,花費1130秒
已完成3.85%分析-1950項,花費1159秒


已完成27.73%分析-14050項,花費7543秒
已完成27.83%分析-14100項,花費7570秒
已完成27.93%分析-14150項,花費7596秒
已完成28.02%分析-14200項,花費7622秒
已完成28.12%分析-14250項,花費7649秒
已完成28.22%分析-14300項,花費7676秒
已完成28.32%分析-14350項,花費7702秒
已完成28.42%分析-14400項,花費7727秒
已完成28.52%分析-14450項,花費7753秒
已完成28.62%分析-14500項,花費7779秒
已完成28.71%分析-14550項,花費7805秒
已完成28.810000000000002%分析-14600項,花費7831秒
已完成28.910000000000004%分析-14650項,花費7857秒
已完成29.01%分析-14700項,花費7882秒
已完成29.110000000000003%分析-14750項,花費7908秒
已完成29.21%分析-14800項,花費7934秒
已完成29.310000000000002%分析-14850項,花費7960秒
已完成29.409999999999997%分析-14900項,花費7986秒
已完成29.5%分析-14950項,花費8012秒
已完成29.599999999999998%分析-15000項,花費8037秒
已完成29.7%分析-15050項,花費8063秒
已完成29.799999999999997%分析-15100項,花費8089秒
已完成29.9%分析-15150項,花費8115秒
已完成30.0%分析-15200項,花費8141秒
已完成30.099999999999998%分析-15250項,花費8167秒
已完成30.19%分析-15300項,花費8192秒
已完成30.29%分析-15350項,花費8218秒
已完成30.39%分析-15400項,花費8244秒
已完成30.490000000000002%分析-15450項,花費8270秒
已完成30.59%分析-15500項,花費8296秒
已完成30.69%分析-15550項,花費8321秒
已完成30.79%分析-15600項,花費8347秒
已完成30.89%分析-15650項,花費83

已完成53.68000000000001%分析-27200項,花費14391秒
已完成53.779999999999994%分析-27250項,花費14417秒
已完成53.879999999999995%分析-27300項,花費14443秒
已完成53.98%分析-27350項,花費14469秒
已完成54.06999999999999%分析-27400項,花費14495秒
已完成54.169999999999995%分析-27450項,花費14521秒
已完成54.269999999999996%分析-27500項,花費14547秒
已完成54.37%分析-27550項,花費14573秒
已完成54.47%分析-27600項,花費14599秒
已完成54.56999999999999%分析-27650項,花費14624秒
已完成54.669999999999995%分析-27700項,花費14652秒
已完成54.769999999999996%分析-27750項,花費14681秒
已完成54.86%分析-27800項,花費14711秒
已完成54.96%分析-27850項,花費14741秒
已完成55.059999999999995%分析-27900項,花費14773秒
已完成55.16%分析-27950項,花費14803秒
已完成55.26%分析-28000項,花費14831秒
No operations to execute
已完成55.36%分析-28050項,花費14858秒
已完成55.46%分析-28100項,花費14885秒
已完成55.55%分析-28150項,花費14911秒
已完成55.65%分析-28200項,花費14938秒
已完成55.75%分析-28250項,花費14964秒
已完成55.85%分析-28300項,花費14990秒
已完成55.95%分析-28350項,花費15016秒
已完成56.05%分析-28400項,花費15042秒
已完成56.15%分析-28450項,花費15068秒
已完成56.25%分析-28500項,花費15095秒
已完成56.34%分析-28550項,花費15121秒
已完成56.44%分析-28600項,花費15146秒
已完成56.54%分析-28650項,花費15172秒
已完成56.64

已完成78.94%分析-40000項,花費21002秒
No operations to execute
已完成79.03999999999999%分析-40050項,花費21029秒
已完成79.14%分析-40100項,花費21055秒
已完成79.24%分析-40150項,花費21081秒
已完成79.34%分析-40200項,花費21108秒
已完成79.43%分析-40250項,花費21134秒
已完成79.53%分析-40300項,花費21160秒
已完成79.63%分析-40350項,花費21186秒
已完成79.73%分析-40400項,花費21213秒
已完成79.83%分析-40450項,花費21239秒
已完成79.93%分析-40500項,花費21265秒
已完成80.03%分析-40550項,花費21291秒
已完成80.12%分析-40600項,花費21317秒
已完成80.22%分析-40650項,花費21343秒
已完成80.32000000000001%分析-40700項,花費21369秒
已完成80.42%分析-40750項,花費21395秒
已完成80.52%分析-40800項,花費21423秒
已完成80.62%分析-40850項,花費21451秒
已完成80.72%分析-40900項,花費21480秒
已完成80.82000000000001%分析-40950項,花費21510秒
已完成80.91000000000001%分析-41000項,花費21539秒
已完成81.01%分析-41050項,花費21566秒
已完成81.11%分析-41100項,花費21593秒
已完成81.21000000000001%分析-41150項,花費21621秒
已完成81.31%分析-41200項,花費21650秒
已完成81.41000000000001%分析-41250項,花費21678秒
已完成81.51%分析-41300項,花費21706秒
已完成81.6%分析-41350項,花費21733秒
已完成81.69999999999999%分析-41400項,花費21762秒
已完成81.8%分析-41450項,花費21789秒
已完成81.89999999999999%分析-41500項,花費21817秒
已完成82.0%分析-41

In [10]:
import pandas

df=pandas.DataFrame(TWLocs)

writer=pandas.ExcelWriter(r"D:\outputXLS\ita.xlsx")

sa=df.to_excel(writer)

writer.save()